In [1]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time
import pandas as pd


class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self,self)
        self.pos_df = pd.DataFrame(columns=['Account', 'Symbol', 'SecType',
                                            'Currency', 'Position', 'Avg cost'])
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId) 
        
    def position(self, account, contract, position, avgCost):
        super().position(account, contract, position, avgCost)
        dictionary = {"Account":account, "Symbol": contract.symbol, "SecType": contract.secType,
                      "Currency": contract.currency, "Position": position, "Avg cost": avgCost}
        if self.pos_df["Symbol"].str.contains(contract.symbol).any():
            self.pos_df.loc[self.pos_df["Symbol"]==contract.symbol,"Position"] = position
            self.pos_df.loc[self.pos_df["Symbol"]==contract.symbol,"Avg cost"] = avgCost
        else:
            self.pos_df = pd.concat((self.pos_df, pd.DataFrame([dictionary])),ignore_index=True)
            #self.pos_df = self.pos_df.append(dictionary, ignore_index=True)
        

def websocket_con():
    app.run()
    
app = TradingApp()      
app.connect("127.0.0.1", 7497, clientId=1)

# starting a separate daemon thread to execute the websocket connection
con_thread = threading.Thread(target=websocket_con, daemon=True)
con_thread.start()
time.sleep(1) # some latency added to ensure that the connection is established

def usStk(symbol, sectype ="STK", currency = "USD", exchange="NASDAQ"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sectype
    contract.currency = currency
    contract.exchange = exchange
    return contract

#creating object of the Contract class - will be used as a parameter for other function calls
def mktOrder(direction,quantity):
    order = Order()
    order.action = direction
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    return order

app.reqPositions()
time.sleep(1)
pos_df = app.pos_df

app.placeOrder(app.nextValidOrderId,usStk("QCOM"),mktOrder("BUY",4))

app.reqPositions()
time.sleep(1)
pos_df = app.pos_df

ERROR -1 0 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 0 2104 Market data farm connection is OK:hfarm
ERROR -1 0 2104 Market data farm connection is OK:jfarm
ERROR -1 0 2104 Market data farm connection is OK:usfuture
ERROR -1 0 2104 Market data farm connection is OK:cashfarm
ERROR -1 0 2104 Market data farm connection is OK:eufarmnj
ERROR -1 0 2104 Market data farm connection is OK:usfarm
ERROR -1 0 2106 HMDS data farm connection is OK:euhmds
ERROR -1 0 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 0 2106 HMDS data farm connection is OK:ushmds
ERROR -1 0 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 20


ERROR 20 0 399 Order Message: BUY 4 QCOM NASDAQ.NMS Warning: Your order will not be placed at the exchange until 2025-06-25 09:30:00 US/Eastern.
ERROR 20 0 202 Order Canceled - reason:
ERROR -1 0 1100 Connectivity between IBKR and Trader Workstation has been lost.
ERROR -1 0 2103 Market data farm connection is broken:eufarmnj
ERROR -1 0 2103 Market data farm connection is broken:usfuture
ERROR -1 0 2157 Sec-def data farm connection is broken:secdefil
ERROR -1 0 2103 Market data farm connection is broken:jfarm
ERROR -1 0 2103 Market data farm connection is broken:hfarm
ERROR -1 0 2105 HMDS data farm connection is broken:fundfarm
ERROR -1 0 2103 Market data farm connection is broken:usfarm.nj
ERROR -1 0 2105 HMDS data farm connection is broken:ushmds
ERROR -1 0 2103 Market data farm connection is broken:usfarm
ERROR -1 0 2105 HMDS data farm connection is broken:euhmds
ERROR -1 0 2104 Market data farm connection is OK:jfarm
ERROR -1 0 2104 Market data farm connection is OK:hfarm
ERROR -1 

In [2]:
app.disconnect()